In [1]:
!pip install optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 296kB 18.4MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 163kB 55.7MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 143kB 54.4MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 81kB 13.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=02fd0585348b2e6d39cffffe90796b532ecb9807cb8af18200702764db764983
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [67]:
train = pd.read_csv('/content/drive/MyDrive/DACON/credit/train.csv')
train = train.drop(['index'], axis=1)

test = pd.read_csv('/content/drive/MyDrive/DACON/credit/test.csv')
test = test.drop(['index'], axis=1)

submit = pd.read_csv('/content/drive/MyDrive/DACON/credit/sample_submission.csv')

In [68]:
train= train.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)
test= test.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)

In [44]:
test

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month
0,N,112500.0,Pensioner,-21990,365243,NaN,-60.0
1,Y,135000.0,State servant,-18964,-8671,Core staff,-36.0
2,Y,69372.0,Working,-15887,-217,Laborers,-40.0
3,N,112500.0,Commercial associate,-19270,-2531,Drivers,-41.0
4,Y,225000.0,State servant,-17822,-9385,Managers,-8.0
...,...,...,...,...,...,...,...
9995,Y,202500.0,Working,-18593,-5434,Accountants,-19.0
9996,Y,202500.0,Working,-10886,-1315,Laborers,-34.0
9997,Y,292500.0,Working,-21016,-14018,Medicine staff,-55.0
9998,N,180000.0,Commercial associate,-16541,-1085,NaN,-33.0


In [45]:
def occyptype_pre(data):
  data.loc[data["DAYS_EMPLOYED"] >= 0,'occyp_type'] = "No job"
  data["occyp_type"] = data["occyp_type"].fillna("unknown")
  return

In [69]:
occyptype_pre(train)
occyptype_pre(test)

In [47]:
#Binary variables
def binary(data):
  print('Having house reality or not: ')
  data['reality'] = data['reality'].replace(['N','Y'],[0,1])
  print(data['reality'].value_counts())
  print('--------------')


In [70]:
binary(train)

Having house reality or not: 
1    17830
0     8627
Name: reality, dtype: int64
--------------


In [71]:
binary(test)

Having house reality or not: 
1    6676
0    3324
Name: reality, dtype: int64
--------------


In [72]:
def preprocess(data):
  label_encoder = LabelEncoder()
  data['income_type']=label_encoder.fit_transform(data['income_type'])
  data['occyp_type']=label_encoder.fit_transform(data['occyp_type'])
  return

In [73]:
preprocess(train)
preprocess(test)

In [52]:
train

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month,credit
0,0,202500.0,0,-13899,-4709,19,-6.0,1.0
1,1,247500.0,0,-11380,-1540,8,-5.0,1.0
2,1,450000.0,4,-19087,-4434,10,-22.0,2.0
3,1,202500.0,0,-15088,-2092,15,-37.0,0.0
4,1,157500.0,2,-15037,-2105,10,-26.0,2.0
...,...,...,...,...,...,...,...,...
26452,0,225000.0,2,-12079,-1984,3,-2.0,1.0
26453,1,180000.0,4,-15291,-2475,19,-47.0,2.0
26454,0,292500.0,4,-10082,-2015,3,-25.0,2.0
26455,1,171000.0,4,-10145,-107,8,-59.0,2.0


### Normalization

In [74]:
continuos_variable = ['income_total','DAYS_BIRTH', 'DAYS_EMPLOYED','begin_month']
train_continuos = train[continuos_variable]
test_continuos = test[continuos_variable]
train_continuos_copy = train[continuos_variable]
test_continuos_copy = test[continuos_variable]

In [75]:
train_continuos_copy.columns

Index(['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month'], dtype='object')

In [76]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler = scaler.fit(train_continuos)
train_continuos_scaled = scaler.transform(train_continuos)
test_continuos_scaled = scaler.transform(test_continuos)

train_df_continuos_scaled = pd.DataFrame(train_continuos_scaled, columns=train_continuos_copy.columns, index=list(train_continuos_copy.index.values))
test_df_continuos_scaled = pd.DataFrame(test_continuos_scaled, columns=test_continuos_copy.columns, index=list(test_continuos_copy.index.values))


In [77]:
train_df_continuos_scaled

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0.434783,0.235934,-1.154406,0.666667
1,0.869565,0.596564,-0.000364,0.703704
2,2.826087,-0.506800,-1.054261,0.074074
3,0.434783,0.065712,-0.201384,-0.481481
4,0.000000,0.073014,-0.206118,-0.074074
...,...,...,...,...
26452,0.652174,0.496492,-0.162054,0.814815
26453,0.217391,0.036650,-0.340859,-0.851852
26454,1.304348,0.782391,-0.173343,-0.037037
26455,0.130435,0.773372,0.521486,-1.296296


In [78]:
train = train.drop(columns=continuos_variable)
test = test.drop(columns=continuos_variable)
train = pd.concat([train,train_df_continuos_scaled], axis = 1)
test = pd.concat([test,test_df_continuos_scaled], axis = 1)

In [79]:
train

,reality,income_type,occyp_type,credit,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0,0,19,1.0,0.434783,0.235934,-1.154406,0.666667
1,1,0,8,1.0,0.869565,0.596564,-0.000364,0.703704
2,1,4,10,2.0,2.826087,-0.506800,-1.054261,0.074074
3,1,0,15,0.0,0.434783,0.065712,-0.201384,-0.481481
4,1,2,10,2.0,0.000000,0.073014,-0.206118,-0.074074
...,...,...,...,...,...,...,...,...
26452,0,2,3,1.0,0.652174,0.496492,-0.162054,0.814815
26453,1,4,19,2.0,0.217391,0.036650,-0.340859,-0.851852
26454,0,4,3,2.0,1.304348,0.782391,-0.173343,-0.037037
26455,1,4,8,2.0,0.130435,0.773372,0.521486,-1.296296


In [57]:
test

,reality,income_type,occyp_type,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0,1,12,-0.434783,-0.922405,133.569556,-1.333333
1,1,2,3,-0.217391,-0.489191,-2.597232,-0.444444
2,1,4,8,-0.851478,-0.048676,0.481428,-0.592593
3,0,0,4,-0.434783,-0.532999,-0.361253,-0.629630
4,1,2,10,0.652174,-0.325698,-2.857247,0.592593
...,...,...,...,...,...,...,...
9995,1,4,0,0.434783,-0.436077,-1.418427,0.185185
9996,1,4,8,0.434783,0.667287,0.081573,-0.370370
9997,1,4,11,1.304348,-0.782963,-4.544428,-1.148148
9998,0,0,19,0.217391,-0.142305,0.165331,-0.333333


In [80]:
X=train.drop('credit',axis=1)
y=train['credit']

In [59]:
def objective(trial,train=train,target=y):

  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

  param={
      'max_depth':trial.suggest_int('max_depth',5,10),
      'learning_rate':trial.suggest_categorical('learning_rate',[0.006,0.008,0.01,0.014,0.017,0.02]),
      'subsample':trial.suggest_categorical('subsample',[0.6,0.7,0.8,0.9,1]),
      'colsample_bytree':trial.suggest_categorical('colsample_bytree',[0.6,0.7,0.8,0.9,1]),
      'eval_metric': 'mlogloss',
      'objective' : 'multi:softprob',
      'random_state': 42,
      'n_estimators': 1000
  }

  model=XGBClassifier(**param)

  model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=30,verbose=False)

  predictions=model.predict_proba(X_test)

  logloss = log_loss(y_test,predictions)
  return logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
study.best_params

[I 2021-05-01 08:52:15,404] A new study created in memory with name: no-name-950a77fa-bf90-4410-8111-55ff0adce97d
[I 2021-05-01 08:53:35,810] Trial 0 finished with value: 0.7482569451082726 and parameters: {'max_depth': 10, 'learning_rate': 0.006, 'subsample': 0.7, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.7482569451082726.
[I 2021-05-01 08:54:35,521] Trial 1 finished with value: 0.7253293484406798 and parameters: {'max_depth': 10, 'learning_rate': 0.02, 'subsample': 0.7, 'colsample_bytree': 0.8}. Best is trial 1 with value: 0.7253293484406798.
[I 2021-05-01 08:55:28,596] Trial 2 finished with value: 0.7812670406006215 and parameters: {'max_depth': 7, 'learning_rate': 0.006, 'subsample': 1, 'colsample_bytree': 0.9}. Best is trial 1 with value: 0.7253293484406798.
[I 2021-05-01 08:56:15,693] Trial 3 finished with value: 0.7473458790045038 and parameters: {'max_depth': 8, 'learning_rate': 0.017, 'subsample': 1, 'colsample_bytree': 0.6}. Best is trial 1 with value: 0.7253293

{'colsample_bytree': 0.6,
 'learning_rate': 0.02,
 'max_depth': 10,
 'subsample': 0.9}

In [81]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [82]:
import random
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = XGBClassifier(n_estimators=1000,
                        objective='multi:softprob',
                        eval_metric='mlogloss',
                        learning_rate=0.02,
                        max_depth=10,
                        subsample=0.9,
                        colsample_bytree=0.6
                        )
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
[0]	validation_0-mlogloss:1.09065	validation_1-mlogloss:1.09133
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 30 rounds.
[100]	validation_0-mlogloss:0.741087	validation_1-mlogloss:0.79695
[200]	validation_0-mlogloss:0.658329	validation_1-mlogloss:0.75667
[300]	validation_0-mlogloss:0.608499	validation_1-mlogloss:0.739813
[400]	validation_0-mlogloss:0.571283	validation_1-mlogloss:0.729738
[500]	validation_0-mlogloss:0.539654	validation_1-mlogloss:0.722525
[600]	validation_0-mlogloss:0.513377	validation_1-mlogloss:0.716977
[700]	validation_0-mlogloss:0.489183	validation_1-mlogloss:0.713231
[800]	validation_0-mlogloss:0.465717	validation_1-mlogloss:0.710398
[900]	validation_0-mlogloss:0.444634	validation_1-mlogloss:0.707895
[999]	validation_0-mlogloss:0.42635	validation_1-mlogloss:0.70649


=========

In [83]:
test.rename(columns = {'reality' : 'f0','income_total' : 'f1', 'income_type' : 'f2', 'DAYS_BIRTH' : 'f3', 'DAYS_EMPLOYED' : 'f4', 'occyp_type' : 'f5', 'begin_month' : 'f6'}, inplace = True)

In [84]:
test = test.reindex(['f0','f1','f2','f3','f4','f5','f6'], axis =1)
test

,f0,f1,f2,f3,f4,f5,f6
0,0,-0.434783,1,-0.922405,133.569556,12,-1.333333
1,1,-0.217391,2,-0.489191,-2.597232,3,-0.444444
2,1,-0.851478,4,-0.048676,0.481428,8,-0.592593
3,0,-0.434783,0,-0.532999,-0.361253,4,-0.629630
4,1,0.652174,2,-0.325698,-2.857247,10,0.592593
...,...,...,...,...,...,...,...
9995,1,0.434783,4,-0.436077,-1.418427,0,0.185185
9996,1,0.434783,4,0.667287,0.081573,8,-0.370370
9997,1,1.304348,4,-0.782963,-4.544428,11,-1.148148
9998,0,0.217391,0,-0.142305,0.165331,19,-0.333333


In [85]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [87]:
submit.to_csv('xgb_v2.csv', index=False) 